In [1]:
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
import torch

device = torch.device('cpu')

model_pth = "checkpoints/checkpoint-126000/"

model = VisionEncoderDecoderModel.from_pretrained(model_pth)
tokenizer = AutoTokenizer.from_pretrained(model_pth)
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

model.to(device)

"""
华俊龙苦笑着说：文晋，这不公平：你的
都教一个跳，我却教两个班，还是二年级，未
免有失公道吧！
李文晋笑着说：“是不公平，这分工作总得
有人去干吧？你说，该给谁干呢？我没说一句，
他们就告诉我：这是教育科的安排。廖组长还
说：“你们一起来的华俊龙考师，领导者得他起，
就把重担交给他了。”
张厚生笑着说：“这叫能者多劳，你就担起
这个危险吧，有什么困难，我们都会帮忙的。”
周宏义也说：“我们刚来，学校安排工作
我们还是愉快地接受下来，不说什么为好。当
然俊龙一个人任二年级的课，而且教两个班，
比我们辛苦许多，从另一方面说，也是学校
对你的重视，不过，无论怎么说，还是辛苦老
"""

"""
华俊龙苦笑着说：文晋，这不公平：你的
都教一个跳，我有教两个教：还是二年级，半
免有失公道理吧！
李文晋笑着说：是不公平·这分二作总得
有人去干吧？你说，该经纶干呢？我没说一句，
他的就告诉出：这是教育科的安排心疼但长还
说：你们一起来自华俊龙考师：领导者得他想
孔包含粗皮线他3。
张厚生笑着说：这种稻香变芽，你就把老
这个危险吧·有什么困难·我们都会帮忙好。
周宏义也说：我们刚翠·李松姿那二作
划的还是瞬快地接受下来：不说什么为好。当
丝缠在一个人任二年级好课，而且教两个班
形式仍会幸苦许多：从另一方面说：也是学校
对你的重视：不过。无论怎么说，还是辛苦
"""

test_dir = "test-page"
for file in os.listdir(test_dir):
    if not file.endswith((".jpg", ".png")):
        continue

    image = Image.open(os.path.join(test_dir, file)).convert("RGB")

    pixel_values = (processor(image, return_tensors="pt").pixel_values).to(device)
    generated_ids = model.generate(pixel_values)
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(generated_text.replace(" ", ""))

# text = "李文晋就走来了。他告诉大家说：“刚才我从廖"
# image = Image.open(os.path.join("sample_imgs", "0.png")).convert("RGB")
# # print(processor.image_processor.to_dict())
# pixel_values = processor(image, return_tensors="pt").pixel_values
# labels = tokenizer(text, return_tensors="pt").input_ids
# # outputs = model(pixel_values, labels=labels)
# outputs = model.generate(pixel_values)
# generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# print(labels, outputs, generated_text)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/home/rocm-user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


华俊龙苦笑着说：文晋，这不公平：你的
都教一个班，我都教两个班：还是二年级，半
免有失公道吧吧！
李文晋笑着说：「是不公平：这台工作总得
有人去干吧？你说，该给谁干呢？外谈说一句，
他们就告诉我：这是教弟科的要那儿廖绍长还


KeyboardInterrupt: 

In [7]:
import os
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from PIL import Image
import torch

device = torch.device('cpu')

model_pth = "uer/roberta-base-word-chinese-cluecorpussmall"

tokenizer = AutoTokenizer.from_pretrained(model_pth)

tokenizer("你是我的小苹果 你是我的大苹果")
# tokenizer.decode([2, 17, 7158, 8884, 8884, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

{'input_ids': [2, 17, 71192, 84, 1206, 17, 71192, 59, 1206, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [2]:
import evaluate
import numpy as np

cer_metric = evaluate.load("cer")
wer_metric = evaluate.load("wer")


def compute_metrics(pred):
    labels_ids = pred.get("label_ids")
    pred_ids = pred.get("predictions")

    # Predictions come with space between
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    labels_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    print(labels_str, pred_str)

    cer = cer_metric.compute(predictions=pred_str, references=labels_str)
    wer = wer_metric.compute(predictions=pred_str, references=labels_str)

    return {"cer": cer, "wer": wer}


pred = {
    "predictions": np.array([outputs]),
    "label_ids": np.array([labels])
}

compute_metrics(pred)

Using the latest cached version of the module from /home/rocm-user/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--wer/85bee9e4216a78bb09b2d0d500f6af5c23da58f9210e661add540f5df6630fcd (last modified on Sat May 20 10:39:44 2023) since it couldn't be found locally at evaluate-metric--wer, or remotely on the Hugging Face Hub.


NameError: name 'outputs' is not defined

In [6]:
import os
from os import path

import torch
from torch.utils.data import Dataset
import torchvision.io as io
import pandas as pd
from PIL import Image

from data_aug import build_data_aug
from torch.utils.data import Subset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import TrOCRProcessor, VisionEncoderDecoderModel, AutoTokenizer
from transformers import default_data_collator

model = VisionEncoderDecoderModel.from_pretrained("checkpoints/checkpoint-360000/")
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-base")
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")


class OCRDataset(Dataset):

    def __init__(self,
                 dataset_dir,
                 labels_dir,
                 transform,
                 processor,
                 tokenizer,
                 mode="train",
                 max_target_length=32,
                 device=None):
        self.dataset_dir = dataset_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.device = device
        self.processor = processor
        self.mode = mode
        self.max_target_length = max_target_length
        self.tokenizer = tokenizer
        self.df = self.build_df()
        self.df_len = len(self.df)
        self.arbitrary_len = 12000000

    def __len__(self):
        return self.arbitrary_len

    def __getitem__(self, idx):

        # Thirty percent of the time, use existing dataset
        if idx < self.df_len:
            text = self.df['text'][idx]
            # get file name + text
            file_name = self.df["file_name"][idx]
            # prepare image (i.e. resize + normalize)
            image = Image.open(path.join(self.dataset_dir,
                                         file_name)).convert("RGB")
        # 70% percent of the time, use online generated data
        else:
            text_idx = int(idx / self.arbitrary_len * (self.df_len - 1))
            text = self.df['text'][text_idx]
            bgr_image = synthesize(text)
            rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(rgb_image)

        if self.transform:
            image = self.transform(image)

        pixel_values = self.processor(image, return_tensors="pt").pixel_values

        # Remove spaces from text, as only the tang-syn version 1 had spaces before text
        text = text.strip()
        labels = self.tokenizer(text,
                                padding="max_length",
                                stride=32,
                                truncation=True,
                                max_length=self.max_target_length).input_ids

        # important: make sure that PAD tokens are ignored by the loss function
        labels = [
            label if label != self.tokenizer.pad_token_id else -100
            for label in labels
        ]

        encoding = {
            "pixel_values": pixel_values.squeeze(),
            "labels": torch.tensor(labels)
        }
        return encoding

    def build_df(self):
        li = []
        for root, _dirs, files in os.walk(self.labels_dir):
            for file in files:  # Loop through the dataset tsvfiles
                if not file.endswith(".tsv"):
                    continue

                print(f"Processing {file}")
                li.append(
                    pd.read_table(path.join(root, file),
                                  names=["file_name", "text"]))

        return pd.concat(li, axis=0, ignore_index=True)
    

dataset_dir = 'dataset/data'
max_length = 64

# Define the number of samples to keep in eval dataset
num_samples = 200

eval_dataset = OCRDataset(
    dataset_dir=dataset_dir,
    labels_dir="dataset/labels/test",
    tokenizer=tokenizer,
    processor=processor,
    mode="eval",
    transform=None,
    max_target_length=max_length
)

# Create a random subset of the dataset
subset_indices = torch.randperm(len(eval_dataset))[:num_samples]
eval_dataset = Subset(eval_dataset, subset_indices.tolist())

print("Number of validation examples:", len(eval_dataset))


def init_trainer(model, tokenizer, compute_metrics, train_dataset,
                 eval_dataset):
    training_args = Seq2SeqTrainingArguments(
        predict_with_generate=True,
        evaluation_strategy="steps",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        fp16=True,
        learning_rate=4e-5,
        output_dir="./checkpoints",
        logging_dir="./logs",
        logging_strategy="steps",
        logging_steps=100,
        save_strategy="steps",
        save_total_limit=5,
        save_steps=10000,
        eval_steps=10000,
        resume_from_checkpoint="./checkpoints/",
        dataloader_num_workers=10)

    # instantiate trainer
    return Seq2SeqTrainer(
        model=model,
        tokenizer=tokenizer,
        args=training_args,
        compute_metrics=compute_metrics,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=default_data_collator,
    )

trainer = init_trainer(model, tokenizer, compute_metrics, eval_dataset, eval_dataset)

eval_result = None
with torch.no_grad():
    eval_result = trainer.evaluate(eval_dataset, max_length=64)

/home/dotin13/mijo/GitHub/hand-syn/tang-syn/venv/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/dotin13/mijo/GitHub/hand-syn/tang-syn/venv/lib/python3.11/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to u

Processing hwdb2.0_4k.tsv
Processing hwdb_ic13_3k.tsv
Processing hw_chinese_17k.tsv
Processing hwdb2.2_3k.tsv
Processing hwdb2.1_3k.tsv
Processing hwdb_ic13_val_5k.tsv
Number of validation examples: 200


/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory
/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory


In [7]:
eval_result

{'eval_loss': 0.3815649747848511,
 'eval_cer': 0.09589737076082815,
 'eval_wer': 0.16901408450704225,
 'eval_runtime': 23.1828,
 'eval_samples_per_second': 8.627,
 'eval_steps_per_second': 0.561}

In [ ]:
import pstats
from pstats import SortKey
p = pstats.Stats('0.log')
p.strip_dirs().sort_stats("cumtime").print_stats()

In [49]:
# Cleanup fonts that does not support cjk

import os
import pygame
import pygame.freetype
from fontTools.ttLib import TTFont

pygame.freetype.init()


def get_font_names(font_path):
    font = TTFont(font_path)
    names = []
    for record in font['name'].names:
        if record.nameID == 4:
            if record.getEncoding() == "x_mac_simp_chinese_ttx":
                names.append(record.string.decode('gbk'))
            else:
                names.append(record.toStr())

    return names


for file in os.listdir("fonts"):
    if file.lower().endswith((".ttf", ".otf")):
        # font = pygame.freetype.Font(f"fonts/{file}", size=10)
        # metrics = font.get_metrics("你是我的小苹果，我是你的大苹果！")

        # invalid_metrics = not all(metrics)

        names = get_font_names(f"fonts/{file}")
        invalid_type = any("篆" in font_name or (
            "繁" in font_name and "简" not in font_name) for font_name in names)

        if invalid_type:
            # print(names)
            os.rename(f"fonts/{file}", f"fonts/archives/{file}")